# Наработки

In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [19]:
def convert_from_bin_to_pcd(path_to_binary_file : str, path_to_new_pcd_file : str):
    bin_pcd = np.fromfile(path_to_binary_file, dtype=np.float32)
    points = bin_pcd.reshape((-1, 4))[:, 0:3]
    o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
    o3d.io.write_point_cloud(path_to_new_pcd_file, o3d_pcd)

In [20]:
def convert_from_pcd_to_ply(path_to_pcd_file : str, path_to_new_ply_file : str):
    pcd = o3d.io.read_point_cloud(path_to_pcd_file)
    o3d.io.write_point_cloud(path_to_new_ply_file, pcd)

In [21]:
def scan_next_plane_from_point_cloud(point_cloud : o3d.geometry.PointCloud, distance : float = 0.1):
    try:
        _, inliers = point_cloud.segment_plane(distance_threshold=distance,
                                             ransac_n=3,
                                             num_iterations=5000)
    except Exception:
        return (point_cloud, point_cloud.clear())
    inlier_cloud = point_cloud.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])
    outlier_cloud = point_cloud.select_by_index(inliers, invert=True)
    return (inlier_cloud, outlier_cloud)

In [22]:
def select_points_by_label_id(path_to_pcd_file : str, path_to_label_file : str, label_id : int) -> o3d.geometry.PointCloud:
    pcd = o3d.io.read_point_cloud(path_to_pcd_file)

    labels = np.fromfile(path_to_label_file, dtype=np.uint32)
    labels = labels.reshape((-1))
    
    pcd_point_by_id = pcd.select_by_index(np.where(labels == label_id)[0])
    
    return pcd_point_by_id

In [23]:
def segment_all_planes_from_point_cloud(point_cloud : o3d.geometry.PointCloud) -> list:
    all_planes = []
    inlier_cloud, outlier_cloud = scan_next_plane_from_point_cloud(point_cloud)
    while outlier_cloud.has_points():
        all_planes.append(inlier_cloud)
        inlier_cloud, outlier_cloud = scan_next_plane_from_point_cloud(outlier_cloud)
    if (inlier_cloud.has_points()): all_planes.append(inlier_cloud)
    
    return all_planes

In [24]:
def segment_all_planes_by_list_of_labels(path_to_pcd_file : str, path_to_label_file : str, list_of_labels : list) -> dict:
    result_dict = {}
    for label in list_of_labels:
        point_cloud = select_points_by_label_id(path_to_pcd_file, path_to_label_file, label)
        planes = extract_all_planes_from_point_cloud(point_cloud)
        result_dict[label] = planes
    
    return result_dict

In [25]:
path_to_bin_file = "/home/pavel/dataset/sequences/00/velodyne/000000.bin"
path_to_pcd_file = "/home/pavel/Point-Cloud/src/test.pcd"
path_to_label_file = "/home/pavel/dataset/sequences/00/labels/000000.label"

In [26]:
BUILDING_LABEL = 50
OTHER_STRUCTURE_LABEL = 52
ROAD_LABEL = 40
list_of_planes = [BUILDING_LABEL, OTHER_STRUCTURE_LABEL, ROAD_LABEL]

In [27]:
segmented_planes = segment_all_planes_by_list_of_labels(path_to_pcd_file, path_to_label_file, list_of_planes)

In [28]:
len(segmented_planes)

3